In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import seaborn as sns; sns.set(style="ticks", palette='Set1')
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import scikitplot as skplt
import seaborn as sns
import scipy

In [2]:
from lightfm import LightFM

In [86]:
train = pd.read_csv('train.csv', delimiter=',', header = 0)
test = pd.read_csv('test.csv', delimiter=',', header = 0)
user_features = pd.read_csv('user-features.csv', delimiter = ',', header = 0)
item_features = pd.read_csv('item-features.csv', delimiter=',', header = 0)

In [87]:
train = train[train['item_id'] < 209]
train = train.loc[:, [col for col in train.columns if col != 'timestamp']]
item_features = item_features[item_features['item_id'] < 209]
item_features = item_features.sort_values(by=['item_id'])
train.head(5)

,user_id,item_id,like
1,378,172,1
2,150,182,0
3,455,17,0
5,122,96,0
6,216,129,0


In [88]:
'''scaler = StandardScaler()
user_features_scaled = scaler.fit_transform(user_features[user_features.columns[1:32]])
item_features_scaled = scaler.fit_transform(item_features[item_features.columns[1:32]])'''

'''pca = PCA(n_components=15)
user_features_pca = pca.fit_transform(user_features_scaled)
pca = PCA(n_components=15)
item_features_pca = pca.fit_transform(item_features_scaled)'''

#user_features = user_features.round()
#item_features = item_features.round()

'pca = PCA(n_components=15)\nuser_features_pca = pca.fit_transform(user_features_scaled)\npca = PCA(n_components=15)\nitem_features_pca = pca.fit_transform(item_features_scaled)'

In [89]:
'''from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(train, test_size = 0.3, random_state = 42)'''

'from sklearn.model_selection import train_test_split\n\nX_train, X_test = train_test_split(train, test_size = 0.3, random_state = 42)'

In [90]:
n_users = user_features.shape[0]
n_items = item_features.shape[0]
train_arr = train.values
interactions = np.zeros(n_users*n_items).reshape(n_users, n_items)
for line in train_arr:
    if int(line[2]) == 1:
        interactions[line[0]][line[1]] = 1
    else:
        interactions[line[0]][line[1]] = -1
        
'''test_arr = X_test.values
test_interactions = np.ones(n_users*n_items).reshape(n_users, n_items)
for line in test_arr:
    if int(line[2]) == 1:
        interactions[line[0]][line[1]] = 5'''

'test_arr = X_test.values\ntest_interactions = np.ones(n_users*n_items).reshape(n_users, n_items)\nfor line in test_arr:\n    if int(line[2]) == 1:\n        interactions[line[0]][line[1]] = 5'

In [91]:
interactions

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [-1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [92]:
interact = scipy.sparse.csr_matrix(interactions.astype(int))
#test_interact = scipy.sparse.csr_matrix(test_interactions.astype(float))
User_features = scipy.sparse.csr_matrix(user_features.iloc[:,1:].astype(float))
Item_features = scipy.sparse.csr_matrix(item_features.iloc[:,1:].astype(float))

In [122]:
model = LightFM(loss='warp', learning_rate = 0.01, no_components = 32, learning_schedule='adagrad')
    #random_state=2016,
    #learning_rate=0.1,
    #learning_schedule='adagrad')
    #no_components=150,
    #user_alpha=0.000005)

model.fit(interact, user_features = User_features, item_features = Item_features, epochs=50)

In [123]:
test_ids = test['user_id'].tolist()

In [124]:
#model.predict(0, np.arange(n_items), user_features = User_features, item_features=Item_features)

In [125]:
a = []
for user_x in test_ids:
    scores = model.predict(user_x, np.arange(n_items), user_features = User_features, item_features=Item_features)
    score_dict = {i: scores[i] for i in range(len(scores))}
    top_20_items = [k for k, v in sorted(score_dict.items(), key=lambda item: item[1], reverse = True)]
    a.append([user_x] + top_20_items[:20])

In [126]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

print(precision_at_k(model, interact, user_features = User_features, item_features=Item_features, k=20).mean())
#print(precision_at_k(model, test_interact, user_features = User_features, item_features=Item_features, k=10).mean())
#train_auc = auc_score(model, interact, user_features = User_features, item_features=Item_features).mean()

0.08209256


In [127]:
#train_auc

In [128]:
a = np.array(a)
cols = ['user_id'] + [str(i) for i in range(20)]
ans = pd.DataFrame(data = a, columns = cols)
ans.head()

,user_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,166,76,35,37,22,60,65,80,72,66,...,58,7,11,40,148,36,21,5,59,87
1,26,76,35,37,22,60,65,80,72,66,...,58,7,11,40,148,36,21,5,59,87
2,41,76,35,37,22,60,65,80,72,66,...,58,7,11,40,148,36,21,5,59,87
3,286,76,35,37,22,60,65,80,72,66,...,58,7,11,40,148,36,21,5,59,87
4,108,76,35,37,22,60,65,80,72,66,...,58,7,11,40,148,36,21,5,59,87


In [114]:
ans.to_csv('ans_lightfm_1.csv', index = False)